In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import random
import time
import re

In [14]:
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/' + str(word)
    return url

In [15]:
def select_first(driver):
    first = driver.find_elements_by_css_selector('div._9AhH0')[0]
    first.click()
    time.sleep(random.randrange(3))

In [16]:
import re
from bs4 import BeautifulSoup
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
 # 본문 내용
    try:
        content = soup.select('div.C4VMK > div.MOdxS ')[0].text
    except:
        content = ' '
 # 해시태그
    tags = re.findall(r'#[^\s#,\\]+', content)
 # 작성일자
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
 # 좋아요
    try:
        like = soup.select('a.zV_Nj')[0].text[4:-1] #2021/08/20 수정
    except:
        like = 0
 # 위치
    try:
         place = soup.select('div.M30cS')[0].text
    except:
         place = ''
    data = [content, date, like, place, tags]
    return data

In [17]:
def move_next(driver):
    right = driver.find_element_by_css_selector('div.l8mY4.feth3')
    right.click()
    time.sleep(3)

In [18]:
# 크롤링 시작
"""
driver.get(url)을 통해 검색 페이지 접속하고,
target 변수에 크롤링할 게시글의 수를 바인딩
"""
# 크롬 브라우저 열기
driver = webdriver.Chrome('chromedriver.exe')

driver.get('https://www.instagram.com')
time.sleep(3)

# 인스타그램 로그인을 위한 계정 정보
email = ''
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = ''
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

# 게시물을 조회할 검색 키워드 입력 요청
word = input("검색어를 입력하세요 : ")
word = str(word)
url = insta_searching(word)

# 검색 결과 페이지 열기
driver.get(url)
time.sleep(8) #대기시간
# time.sleep(random.randrange(25,35)) # 코드 수행 환경에 따라 페이지가 로드되는 데 시간이 더 걸릴 수 있어 8초로 변경(2022/01/11)
# 첫 번째 게시물 클릭
select_first(driver)
# 본격적으로 데이터 수집 시작
results = []
## 수집할 게시물의 수
target = 3
for i in range(target):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
        
print(results[:2])

C:\Users\user\AppData\Local\Temp/ipykernel_15764/1183013933.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
C:\Users\user\AppData\Local\Temp/ipykernel_15764/1183013933.py:14: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
C:\Users\user\AppData\Local\Temp/ipykernel_15764/1183013933.py:19: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]


검색어를 입력하세요 : 지속가능한삶


C:\Users\user\AppData\Local\Temp/ipykernel_15764/1390256566.py:2: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  first = driver.find_elements_by_css_selector('div._9AhH0')[0]
C:\Users\user\AppData\Local\Temp/ipykernel_15764/2127103617.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  right = driver.find_element_by_css_selector('div.l8mY4.feth3')


[]


In [21]:
results_df = pd.DataFrame(results)
results_df.columns = ['content', 'data', 'like', 'place', 'tags']
results_df.to_csv('서연짱.csv')

ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements